<a href="https://colab.research.google.com/github/Guhabrus/Laba_1_CUDA/blob/main/Cuda1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-09-14 03:36:12--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-09-14 03:36:13--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?S3_aucWC2DjiZdsxHyJmy2Oo9DwEVe1n3ozX-0UA0PmIoRcEAGHdbCtdDYXIa7uwXSw3Wgw2hxtG4mdXf6fuDqA9pUImxh4-ytu3RkJwbwzFo

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2iqs2h9g
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2iqs2h9g
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=9c05ce05a73725909c7845ae0f483b483da5aa6dd4c3edbb4960bc3bedbf8ebc
  Stored in directory: /tmp/pip-ephem-wheel-cache-e76t5zc8/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [17]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include "device_functions.h"
#include <stdio.h>
#include <fstream>
//#include "../common/cpu_bitmap.h"





const int Count = 65532;                           //кол-во выводимых чисел

__global__ void printMessagdeGPU(int *a)          //Функция выполняема на GPU
{
    int Value = threadIdx.x + blockIdx.x * blockDim.x ;
    if(Value <= Count)
    {
        printf("ValueGPU = %d;\n ", a[Value]);
    }
    
}

void printMessageCPU()                      //Функция выполняемая на CPU
{
    for(int i=1;i<Count; i++)
    {
        printf("ValueCPU = %d;\n", i);
    }
}


int main(void)
{


    int a[Count];
    int *dev_a;
    for(int i=0; i<Count;i++)
    {
        a[i] = i;
    }
    cudaMalloc((void**)&dev_a, Count*sizeof(int));
 
    cudaMemcpy(dev_a, a , Count*sizeof(int), cudaMemcpyHostToDevice);

	cudaEvent_t start, stop;
	float gpu_time = 0.0;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
 
    printMessagdeGPU<<< (Count+1023)/1024 , 1024 >>>(dev_a);
 
    cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	
	cudaEventDestroy(start);
	cudaEventDestroy(stop);
    printf("time = %2fmiliseconds;",gpu_time);
   
    cudaFree(dev_a);


	return 0;
}

Выходные данные были обрезаны до нескольких последних строк (5000).
ValueGPU = 60533;
ValueGPU = 60534;
ValueGPU = 60535;
ValueGPU = 60536;
ValueGPU = 60537;
ValueGPU = 60538;
ValueGPU = 60539;
ValueGPU = 60540;
ValueGPU = 60541;
ValueGPU = 60542;
ValueGPU = 60543;
ValueGPU = 60544;
ValueGPU = 60545;
ValueGPU = 60546;
ValueGPU = 60547;
ValueGPU = 60548;
ValueGPU = 60549;
ValueGPU = 60550;
ValueGPU = 60551;
ValueGPU = 60552;
ValueGPU = 60553;
ValueGPU = 60554;
ValueGPU = 60555;
ValueGPU = 60556;
ValueGPU = 60557;
ValueGPU = 60558;
ValueGPU = 60559;
ValueGPU = 60560;
ValueGPU = 60561;
ValueGPU = 60562;
ValueGPU = 60563;
ValueGPU = 60564;
ValueGPU = 60565;
ValueGPU = 60566;
ValueGPU = 60567;
ValueGPU = 60568;
ValueGPU = 60569;
ValueGPU = 60570;
ValueGPU = 60571;
ValueGPU = 60572;
ValueGPU = 60573;
ValueGPU = 60574;
ValueGPU = 60575;
ValueGPU = 60576;
ValueGPU = 60577;
ValueGPU = 60578;
ValueGPU = 60579;
ValueGPU = 60580;
ValueGPU = 60581;
ValueGPU = 60582;
ValueGPU = 60583;
ValueGPU = 605